In [8]:
import hostPopulation

In [7]:
time = hostPopulation(0, 10, 10)

class Instance():

    def __init__():
        populations
        populationValues
        timeValues

TypeError: 'module' object is not callable